# **Exploratory Data Analysis (EDA)**

## Objectives
The objective is to predict the survival of coronary artery disease patients using the dataset
provided to help doctors to formulate preemptive medical treatments. In your submission, you
are to evaluate at least 3 suitable models for estimating the patients’ survivals.
 
## Dataset description
The dataset contains the medical records of coronary artery disease patients for a particular
hospital. Do note that there could be synthetic features in the dataset. Hence, please ensure
that you state and verify any assumptions that you make.

## Steps

1. Load data
2. Data exploration and cleaning
   * General cleaning
   * Analysis and re-formatting of data, identifying numerical vs categorical values
   * Separate features and label
   * Examine each feature's distribution for each label
3. Training
   * Split the data between train and test sets
   * Train and Evaluate a Binary Classification Model
4. Metrics Analysis
   * Accuracy
   * Recall
   * Precision
   * F1 Score
   * Analysis and selection of metric
   * Classification Report
   * Confusion Matrix
   * ROC Curve (Receiver Operator Characteristic)
   * AUC (Area Under Curve)
5. Pipeline creation
   * Create a model
   * Get prediction and plot performance
6. Pipeline creation with refactored code
   * Experimentation and analysis of features on predictive ability
   * Experimentation and analysis of different algorithms on predictive ability
7. Conclusion

***
## 1. Load data

In [ ]:
#!/usr/bin/env python3

In [ ]:
# Load SQL data

import os
import sqlite3

# Get relative path
dir = os.getcwd()
fullpath = os.path.join(dir, 'data', 'survive.db') # join subdirectorys with ',' instead of '/' as need to be OS-independant
print(fullpath)

# Connection object
o_conn = sqlite3.connect(fullpath)

# Cursor object (for executing SQL queries against database)
cur = o_conn.cursor()

# List table names
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

In [ ]:
# Load data into dataframe

import pandas as pd

s_tablename = 'survive' # change table name accordingly if not 'survive'
s_SQLquery = 'SELECT * FROM ' + s_tablename

df = pd.read_sql_query(s_SQLquery, o_conn)
df.head()

***
# 2. Data exploration and cleaning

### 2a. General cleaning

In [ ]:
# Check for empty fields
df.isnull().sum()

In [ ]:
# Missing Creatinine records = 499
# Or 499/15000 = 3.3% of records, made subjective call to drop them entirely.
df = df.dropna(subset=['Creatinine'])

# Verify they've been dropped
df.isnull().sum()

In [ ]:
# Quick view of unique values for each column
for column in df:
    a = df[column].unique()
    print(column, ":", a)

*Initial Observations*
* **ID Unique** - ID for each patient
* **Survive** - If the patient survives: 0 = No , 1 = Yes ***['0' '1' 'No' 'Yes']***
* **Gender** - Gender type ***['Male' 'Female']***
* **Smoke** - If the patient smokes ***['Yes' 'No' 'NO' 'YES']***
* **Diabetes** - Diabetes conditions of patient ***['Normal' 'Pre-diabetes' 'Diabetes']***
* **Age** - Age of the patient ***[integers, some negative]***
* **Ejection Fraction** - Strength of heart ***['Low' 'Normal' 'High' 'L' 'N']***
* **Sodium** - Level of sodium in the blood serum ( mg/dL) ***[integers]***
* **Creatinine** - Level of creatinine in the blood serum (mEq/L) ***[floats]***
* **Platelets** - Number of platelets in the blood serum (platelets/mL) ***[mostly integers in the thousands, some floats that look like errors]***
* **Creatine phosphokinase** - Level of the enzyme in the blood (mcg/L) ***[floats]***
* **Blood Pressure** - Level of blood pressure (mmHg) ***[integers]***
* **Hemoglobin** - Level of hemoglobin in the blood (g/dL) ***[floats]***
* **Height** - Height of patient (cm) ***[integers]***
* **Weight** - Weight of patient (kg) ***[integers]***
* **Favourite color** - Favourite color of patient ***['green' 'black' 'white' 'yellow' 'blue' 'red']***

In [ ]:
# Check for duplicates in 'ID' to see if there are more than 1 each.
print(df['ID'].value_counts())

There are duplicates. Check how many there are.

In [ ]:
# Create a dataframe recording counts of duplicates.
df_ID = df['ID'].value_counts().reset_index()
df_ID.columns = ['ID', 'count']

# Then count those with more than 1 instance.
print(len(df_ID[df_ID['count']>1]))

In [ ]:
# There are 825 IDs with more than 1 entry. Need to investigate if they are duplicates or not. Check a sample entry.
print(df[df['ID'] == 'FHKHIH'])

Looks like 4 different individuals, we can't drop based on duplicate ID.

In [ ]:
# df = df.drop_duplicates(subset='ID', keep='first') # This keeps the first entry and drops the others based on ID alone. We can't do this as established above.

# Combine the column values
df['combined'] = ['_'.join(row.astype(str)) for row in df[df.columns[1:]].values]
print(df['combined'].value_counts())

In [ ]:
# df = df.drop_duplicates(['combined']) # We don't need to drop any duplicates as there are none.

df.drop('combined', axis=1, inplace=True)

***
### 2b. Analysis and re-formatting of data, identifying numerical vs categorical values

In [ ]:
print(df['Survive'].unique())
print(df['Survive'].value_counts())

In [ ]:
# Transform strings to 0 and 1
df = df.replace({'Survive' : { '0' : 0, '1' : 1, 'No' : 0, 'Yes' : 1}})
print(df['Survive'].unique())
print(df['Survive'].value_counts())

'Survive' will be the label in the classification model.

In [ ]:
print(df['Gender'].unique())
print(df['Gender'].value_counts())

In [ ]:
# Transform strings to 0 and 1
df = df.replace({'Gender' : { 'Male' : 1, 'Female' : 0}})
print(df['Gender'].unique())
print(df['Gender'].value_counts())

'Gender' being a categorical feature, will be One Hot Encoded later in the classification pipeline.

In [ ]:
print(df['Smoke'].unique())
print(df['Smoke'].value_counts())

In [ ]:
# Transform strings to 0 and 1
df = df.replace({'Smoke' : { 'Yes' : 1, 'No' : 0, 'NO' : 0, 'YES' : 1}})
print(df['Smoke'].unique())
print(df['Smoke'].value_counts())

'Smoke' being a categorical feature, will be One Hot Encoded later in the classification pipeline.

In [ ]:
print(df['Diabetes'].unique())
print(df['Diabetes'].value_counts())

In [ ]:
df = df.replace({'Diabetes' : { 'Normal' : 0, 'Pre-diabetes' : 1, 'Diabetes' : 2}})
print(df['Diabetes'].unique())
print(df['Diabetes'].value_counts())

'Diabetes' being a categorical feature, will be One Hot Encoded later in the classification pipeline.

In [ ]:
print(df['Age'].unique())
print(df['Age'].value_counts())

In [ ]:
df['Age'] = df['Age'].abs() # assume that recorded negative values are meant to positive
print(df['Age'].unique())
print(df['Age'].value_counts())

Plot distribution to have a view.

In [ ]:
import plotter

In [ ]:
# Plot 'Age' distribution
# plotter.histo_boxplot(df, 'Age', 20)
distri_age = plotter.Distribution(df, 'Age', 20)
distri_age.histo_boxplot()

'Age' being a numeric feature, will be scaled later.

In [ ]:
print(df['Ejection Fraction'].unique())
print(df['Ejection Fraction'].value_counts())

In [ ]:
df = df.replace({'Ejection Fraction' : { 'Low' : 0, 'Normal' : 1, 'High' : 2, 'L' : 0, 'N' : 1}})
print(df['Ejection Fraction'].unique())
print(df['Ejection Fraction'].value_counts())

'Ejection Fraction' being a categorical feature, will be One Hot Encoded later in the classification pipeline.

In [ ]:
print(df['Sodium'].unique())
print(df['Sodium'].value_counts())

In [ ]:
# Plot 'Sodium' distribution
# plotter.histo_boxplot(df, 'Sodium', 20)
distri_sodium = plotter.Distribution(df, 'Sodium', 20)
distri_sodium.histo_boxplot()

Exhbits central tendency. 'Sodium' being a numeric feature, will be scaled later.

In [ ]:
print(df['Creatinine'].unique())
print(df['Creatinine'].value_counts())

In [ ]:
# Plot 'Creatinine' distribution
# plotter.histo_boxplot(df, 'Creatinine', 40)
distri_creatinine = plotter.Distribution(df, 'Creatinine', 40)
distri_creatinine.histo_boxplot()

Positive skew distribution. 'Creatinine' being a numeric feature, will be scaled later.

In [ ]:
print(df['Platelets'].unique())
print(df['Platelets'].value_counts())

In [ ]:
# Plot 'Platelets' distribution
# plotter.histo_boxplot(df, 'Platelets', 50)
distri_platelets = plotter.Distribution(df, 'Platelets', 50)
distri_platelets.histo_boxplot()

Normal-looking distribution. However, there's an strange 263358.03 value. Check how many records might have it.

In [ ]:
# Check for duplicates in 'Platelets' to see if there are more than 1 each.
df['Platelets'].value_counts()

Entries with the same 263358.03 are unusually high. This appears to be an errorneous entry.

In [ ]:
num_StrangePlatelets = len(df[df['Platelets']==263358.03]) # number of records with Platelets as 263358.03
num_Records = len(df) # total remaining valid records
pc_StrangePlatelets = num_StrangePlatelets / num_Records
print(pc_StrangePlatelets)

8.35% of records have the same strange platelet count of 236658.03. That's quite large.

Have to decide whether to drop them or round to the nearest thousand to match the format of the rest. Or change to mean value.

It's a very specific error though. If Platelet count is a major factor in prediction, this error will greatly affect the result.

Have an overview of the records with this value.

In [ ]:
print(df[df['Platelets'] == 263358.03])

Each row looks different enough for the same strange Platelet count to be equal. Even though it's 8.35% of the data set, decided to drop these rows as it seems the entry is from human error and the potential effect on prediction is costly.

In [ ]:
rows1 = len(df.index)
print('rows before deletion:',rows1)

# drop rows where 'Platelets' == 263358.03
df = df[df.Platelets != 263358.03] # this is a very specific condition, could consider deleting values not rounded to thousands *000s

rows2 = len(df.index)
print('rows after deletion:',rows2)

print('rows deleted:',rows1 - rows2)

In [ ]:
# Re-plot 'Platelets' distribution
# plotter.histo_boxplot(df, 'Platelets', 50)
distri_platelets = plotter.Distribution(df, 'Platelets', 50)
distri_platelets.histo_boxplot()

Doesn't seem to adversely affect the distribution.
'Platelets' being a numeric feature, will be scaled later.

In [ ]:
print(df['Creatine phosphokinase'].unique())
print(df['Creatine phosphokinase'].value_counts())

In [ ]:
# Plot 'Creatine phosphokinase' distribution
# plotter.histo_boxplot(df, 'Creatine phosphokinase', 40)
distri_cphospho = plotter.Distribution(df, 'Creatine phosphokinase', 40)
distri_cphospho.histo_boxplot()

Heavily positively skewed. 'Creatine phosphokinase' being a numeric feature, will be scaled later.

In [ ]:
print(df['Blood Pressure'].unique())
print(df['Blood Pressure'].value_counts())

In [ ]:
# Plot 'Blood Pressure' distribution
# plotter.histo_boxplot(df, 'Blood Pressure', 120)
distri_bp = plotter.Distribution(df, 'Blood Pressure', 120)
distri_bp.histo_boxplot()

Does not exhibit central tendency. Certain values separated by a somewhat constant distance appear at twice the frequency as the rest of the values.

It might be possible that this is aggregated data from multiple sources and one source is rounding values according to the discrete jumps.

'Blood Pressure' being a numeric feature, will be scaled later.

In [ ]:
print(df['Hemoglobin'].unique())
print(df['Hemoglobin'].value_counts())

In [ ]:
# Plot 'Hemoglobin' distribution
# plotter.histo_boxplot(df, 'Hemoglobin', 40)
distri_haemo = plotter.Distribution(df, 'Hemoglobin', 40)
distri_haemo.histo_boxplot()

Does not exhibit central tendency. 'Hemoglobin' being a numeric feature, will be scaled later.

In [ ]:
print(df['Height'].unique())
print(df['Height'].value_counts())

In [ ]:
# Plot 'Height' distribution
# plotter.histo_boxplot(df, 'Height', 50)
distri_height = plotter.Distribution(df, 'Height', 50)
distri_height.histo_boxplot()

Quite evenly spread out but again we see certain values separated by a somewhat constant distance appearing at twice the frequency as the rest of the values.
It does seem likely this is aggregated data from multiple sources and one source is rounding values according to the discrete jumps. It remains to be seen whether 'Height' is a good predictor of survivability, but if the later tests show it is, this should be investigated further. Possibly by separating this data apart and retesting.
Would have expected a normal-looking distribution for this feature though.
'Height' being a numeric feature, will be scaled later.

In [ ]:
print(df['Weight'].unique())
print(df['Weight'].value_counts())

In [ ]:
# Plot 'Weight' distribution
# plotter.histo_boxplot(df, 'Weight', 120)
distri_weight = plotter.Distribution(df, 'Weight', 120)
distri_weight.histo_boxplot()

Exhibits central tendency. 'Weight' being a numeric feature, will be scaled later.

In [ ]:
print(df['Favorite color'].unique())
print(df['Favorite color'].value_counts())

In [ ]:
df = df.replace({'Favorite color' : { 'green' : 0, 'black' : 1, 'white' : 2, 'yellow' : 3, 'blue' : 4, 'red' : 5}})
print(df['Favorite color'].unique())
print(df['Favorite color'].value_counts())

'Favorite color' being a categorical feature, will be One Hot Encoded later in the classification pipeline.

In [ ]:
# Overview of changes
for column in df:
    a = df[column].unique()
    print(column, ":", a)

We analyse the size of each label.

We will look to predict the label **'Survive'** = **0** for patients who did not survived coronary artery disease, and **1** for patients who did.

In [ ]:
# Firstly we want to see that size of each label is significant, otherwise the Accuracy metric might not be meaningful.
num_survivors = len(df[df['Survive'] == 1])
num_nonsurvivors = len(df[df['Survive'] == 0])

print('Number of survivors:', num_survivors)
print('Number of non-survivors:', num_nonsurvivors)
rows = len(df)
print('Percentage of survivors: {:.1f}'.format(num_survivors/rows*100),"%")
print('Percentage of non-survivors: {:.1f}'.format(num_nonsurvivors/rows*100),"%")

Size of each label is significant but not symmetrical.

In [ ]:
# overview of reformatted data
df.head()

***
### 2c. Separate features and label

In [ ]:
# copy-paste to features below
print(list(df))

In [ ]:
# Separate features and labels
features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
label = 'Survive'
X, y = df[features].values, df[label].values

for n in range(0,4):
    print("Patient", str(n+1), "\n  Features:",list(X[n]), "\n  Label:", y[n])

***
### 2d. Examine each feature's distribution for each label

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
for col in features:
    df.boxplot(column=col, by='Survive', figsize=(6,6))
    plt.title(col)
plt.show()

**Weight** shows markedly different distributions for non-survivors and survivors.

**Hemoglobin**, **Blood Pressure**, **Creatinine**, **Sodium**, **Age** distributions also show some difference.

***

# 3. Training

### 3a. Split the data between train and test sets

We now need to train a classifier so that it finds a relationship between the features and the label values.

In order to test the effectiveness of our training, we need to reserve a portion of the data for testing, so that we can compare the predicted labels with the already known labels of the test set.

We use the **train_test_split** function from **scikit-learn** package to get a statistically random split of training and test data. We will reserve 30% for testing.

In [ ]:
from sklearn.model_selection import train_test_split

# Split data 70%-30% into training set and test set
split_testsize = 0.30
split_seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_testsize, random_state=split_seed)

print ('Training cases: %d\nTest cases: %d' % (X_train.shape[0], X_test.shape[0]))

***
### 3b. Train and Evaluate a Binary Classification Model
We now train our model by fitting the *training features* (**X_train**) to the *training labels* (**y_train**). We'll start with using the *Logistic Regression*, an algorithm for classification.

We also set a *regularization* parameter, which is used to counteract any bias in the sample, and help the model generalize by avoiding *overfitting* the model to the training data.

In [ ]:
# Train with Logistic Regression
from sklearn.linear_model import LogisticRegression

# Set regularization rate
LogReg_RegularizationRate = 0.1

model = LogisticRegression(C=1/LogReg_RegularizationRate, solver="liblinear").fit(X_train, y_train)
print(model)

The model is now trained using the training data.

We now use the test data earlier held back to evaluate how well it predicts.

In [ ]:
# Use the model to predict labels for the test set, and compare the predicted labels to the known labels.
predictions = model.predict(X_test)
print('Predicted labels:', predictions)
print('Actual labels:   ', y_test)

***
# 4. Metrics Analysis

There's too much to ingest visually so we'll examine some metrics with the below:
* Accuracy
* Recall
* Precision
* F1 Score
* Classification Report
* Confusion Matrix
* ROC Curve (Receiver Operator Characteristic)
* AUC (Area Under Curve)

Some definitions first:
* *True Positives* (TP): The predicted label and the actual label are both 1.
* *False Positives* (FP): The predicted label is 1, but the actual label is 0.
* *False Negatives* (FN): The predicted label is 0, but the actual label is 1.
* *True Negatives* (TN): The predicted label and the actual label are both 0.

***
### 4a. Accuracy
* The ratio of *correct* predictions to the *total* number of predictions.
* Accuracy = (TP+TN)/(TP+TN+FP+FN) = (sum of True Positive and True Negative predictions) / (total number of predictions)
* Answers this question: ***Of all predictions, how many are predicted correctly?***
* Value of 1 means 100% of the model's predictions were right, with 0 meaning 100% were wrong.
* It is a good measure if the labels are generally symmetrical, inversely, if the percentage of non-survivors is too small, e.g. 5%, a classifier that always predicts 0 cases of non-survivors will be 95% accurate, i.e. the accuracy of 95% is meaningless.
* Accuracy might not be the best metric as the population of non-survivors(67.6%) is twice that of survivors(32.4%).

In [ ]:
from sklearn.metrics import accuracy_score

sk_accuracy = accuracy_score(y_test, predictions)
print('Accuracy:', sk_accuracy)

The model is 79.05% accurate. Quite good.

***
### 4b. Recall
* The ratio of *correct positive* predictions to the *total correct* predictions.
* Recall = TP/(TP+FN) = (True Positive predictions) / (sum of True Positive and False Negative predictions)
* Answers this question: ***Of all that are actually positive, how many are predicted positive?***
* Recall is particularly useful in cases where the impact of a False Negative is severe, e.g. classifying whether you've been hacked or not, classifying whether an elevator cable is service-worthy

In [ ]:
from sklearn.metrics import recall_score

sk_recall = recall_score(y_test, predictions)
print('Recall:', sk_recall)

The Recall is low. Of the total predictions that are actually 1 (Survived), it only predicted 55.3 % as being 1 (Survived).

***
### 4c. Precision
* The ratio of *correct positive* predictions to the *total positive* predictions.
* Precision = TP/(TP+FP) = (True Positive predictions) / (sum of True Positives and False Positives)
* Answers this question: ***Of all that have been predicted positive, how many are actually positive?***
* A high Precision equates to a low False Positive rate.

In [ ]:
from sklearn.metrics import precision_score

sk_precision = precision_score(y_test, predictions)
print('Precision:', sk_precision)

The model is 73.0% precise.

***
### 4d. F1 Score
* The weighted average of Recall and Precision.
* F1 Score = 2 * (Recall * Precision) / (Recall + Precision)
* F1 is usually more useful than Accuracy, particularly with an uneven label distribution.
* Accuracy works best if false positives and false negatives have similar cost. If they are very different, it is better to look at Recall, Precision, and by extension, F1 Score.

In [ ]:
from sklearn.metrics import f1_score

sk_f1score = f1_score(y_test, predictions) # (2 * sk_recall * sk_precision)/(sk_recall + sk_precision)
print('F1 Score:', sk_f1score)

***
### 4e. Analysis and selection of metric
* Accuracy - we've seen that the populations of non-survivors and survivors are not entirely symmetrical
* Recall = (True Positive predictions) / (sum of True Positive and False Negative predictions)
* Precision = (True Positive predictions) / (sum of True Positives and False Positives)

**If this model is to be used as an *initial diagnosis***:

* We want a *high Recall* (patients who actually are at risk) so that we can send them for further tests.
* We can accept a *lower Precision* if the costs of further tests are not significant.
* We choose **Recall** in this scenario.

**If this model is to be used to decide whether to send the patient for *surgery***:

* We similarly want a *high Recall*, as the cost of a false negative is high (patient does not get life-saving surgery).
* However, we also want *high Precision* as we do not want to send patients for unnecessary surgery. The cost of a false positive is similarly high.
* We choose **F1 Score** in this scenario.

As stated in the objectives, since we are targeting to help doctors formulate preemptive medical treatments, we can accept a lower **Precision**, however, we'll still want a high **Recall**.

***
### 4f. Classification Report

This convenient report includes the above metrics for each class of the label (0 and 1), and also

* *Support*: How many instances of this class are there in the test dataset?

The classification report also includes averages for these metrics, including a weighted average that allows for the imbalance in the number of cases of each class.

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_test, predictions))

***
### 4g. Confusion Matrix

View the confusion matrix for an overview of the prediction distribution.

Note that the correct (*true*) predictions form a diagonal line from top left to bottom right - these figures should be significantly higher than the *false* predictions if the model is any good.

It provides a good snapshot overview of the predictions in relation to the actual labels.

In [ ]:
# TN | FP
# ---+---
# FN | TP

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
print ('Confusion Matrix:\n',cm, '\n')
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]
print('TN:',TN)
print('FP:',FP)
print('FN:',FN)
print('TP:',TP)

By definition a confusion matrix C is such that C[i, j] is equal to the number of observations known to be in group i but predicted to be in group j.

***
### 4h. ROC Curve (Receiver Operator Characteristic)

So far, we've considered the predictions from the model as being either 0 or 1 class labels.

What actually gets predicted by a binary classifier is the probability that the label is true (**P(y)**) and the probability that the label is false (1 - **P(y)**).

A threshold value of 0.5 is used to decide whether the predicted label is a 1 (*P(y) > 0.5*) or a 0 (*P(y) <= 0.5*). You can use the **predict_proba** method to see the probability pairs for each case:

In [ ]:
y_scores = model.predict_proba(X_test)
print(y_scores)

The decision to score a prediction as a 1 or a 0 depends on the threshold to which the predicted probabilities are compared. If we were to change the threshold, it would affect the predictions; and therefore change the metrics in the confusion matrix. A common way to evaluate a classifier is to examine the *true positive rate* (which is another name for recall) and the *false positive rate* for a range of possible thresholds. These rates are then plotted against all possible thresholds to form a chart known as a ***Receiver Operator Characteristic (ROC) chart***, like this:

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

# plot ROC curve
fig = plt.figure(figsize=(6, 6))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

The ROC chart shows the curve of the true and false positive rates for different threshold values between 0 and 1. A perfect classifier would have a curve that goes straight up the left side and straight across the top. The diagonal line across the chart represents the probability of predicting correctly with a 50/50 random prediction.
***
### 4i. AUC (Area Under Curve)
The area under the curve (AUC) is a value between 0 and 1 that quantifies the overall performance of the model. The closer to 1 this value is, the better the model.

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

AUC at 0.8319 is moderately good.
***
# 5. Pipeline creation

We now create a pipeline that does some more preprocessing of the data to make it better for the algorithm to fit a model to it. We'll use 2 here:

- Scaling numeric features so they're on the same scale. This prevents features with large values from producing coefficients that disproportionately affect the predictions.
- Encoding categorical variables. By using a *one hot encoding* technique we can create individual binary (true/false) features for each possible category value.

We then select an algorithm to train a model on the training data.

In [ ]:
df.head()

In [ ]:
# 0 - Gender
# 1 - Smoke
# 2 - Diabetes
# 3 - Age
# 4 - Ejection Fraction
# 5 - Sodium
# 6 - Creatinine
# 7 - Platelets
# 8 - Creatine phosphokinase
# 9 - Blood Pressure
# 10 - Hemoglobin
# 11 - Height
# 12 - Weight
# 13 - Favorite color

# Numeric features -
# Age, Sodium, Creatinine, Platelets, Creatine phosphokinase, Blood Pressure, Hemoglobin, Height, Weight
# [3,5,6,7,8,9,10,11,12]

# Categorical features -
# Gender, Smoke, Diabetes, Ejection Fraction, Favorite color
# [0,1,2,4,13]

***
### 5a. Create a model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Define preprocessing for numeric columns (make them on the same scale)
numeric_features = [3,5,6,7,8,9,10,11,12]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features (encode them)
categorical_features = [0,1,2,4,13]
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Create preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('logregressor', LogisticRegression(C=1/LogReg_RegularizationRate, solver="liblinear"))])

# fit the pipeline to train a logistic regression model on the training set
model = pipeline.fit(X_train, (y_train))
print (model)

In [ ]:
# Purpose of this cell is to generate the print output of the selected features for the 1st subplot in the charts further below

# Dictionary for returning the string of the features
features_dict = {
    0: 'Gender',
    1: 'Smoke',
    2: 'Diabetes',
    3: 'Age',
    4: 'Ejection Fraction',
    5: 'Sodium',
    6: 'Creatinine',
    7: 'Platelets',
    8: 'Creatine phosphokinase',
    9: 'Blood Pressure',
    10: 'Hemoglobin',
    11: 'Height',
    12: 'Weight',
    13: 'Favorite color'
}

# Initialise empty string list for feature names
features_list = []

# Combine int list of categorical and numeric features
# categorical_features = [0, 1, 2, 4, 13]
# numeric_features = [3, 5, 6, 7, 8, 9, 10, 11, 12]
# combined_features = [0, 1, 2, 4, 13, 3, 5, 6, 7, 8, 9, 10, 11, 12]
combined_features = categorical_features + numeric_features

# Generate string list from the int list referencing the dictionary
for item in combined_features:
    features_list.append(features_dict[item])

# Generate a continuous string with each string feature on a new line
features_down = '\n'.join(features_list)
print(features_down)
# print('\n'.join(feature_list))

***
### 5b. Get prediction and plot performance

In [ ]:
import numpy as np

# Get predictions from test data
predictions = model.predict(X_test)
y_scores = model.predict_proba(X_test)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

# Format plots arrangement
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(24, 6))

# Print label and features in 1st subplot
fontsize = 14
padding_left_Attrib = 0.1
padding_left_Value = 0.40
padding_top = 0.96
padding_top_increment = 0.07
ax[0].axis("off")
ax[0].text(padding_left_Attrib, padding_top, 'Algorithm:', size=fontsize, va="top", ha="left")
ax[0].text(padding_left_Value, padding_top, 'Logistic Regression', size=fontsize, va="top", ha="left")
ax[0].text(padding_left_Attrib, padding_top-padding_top_increment, 'Test size:', size=fontsize, va="top", ha="left")
ax[0].text(padding_left_Value, padding_top-padding_top_increment, '{:.2f}'.format(split_testsize), size=fontsize, va="top", ha="left")
ax[0].text(padding_left_Attrib, padding_top-2*padding_top_increment, 'Seed:', size=fontsize, va="top", ha="left")
ax[0].text(padding_left_Value, padding_top-2*padding_top_increment, str(split_seed), size=fontsize, va="top", ha="left")
ax[0].text(padding_left_Attrib, padding_top-3*padding_top_increment, 'Label:', size=fontsize, va='top', ha='left')
ax[0].text(padding_left_Value, padding_top-3*padding_top_increment, 'Survive', size=fontsize, va='top', ha='left')
ax[0].text(padding_left_Attrib, padding_top-4*padding_top_increment, 'Features:', size=fontsize, va='top', ha='left')
ax[0].text(padding_left_Value, padding_top-4*padding_top_increment, features_down, size=fontsize, va='top', ha='left')

# Plot ROC curve in 2nd subplot
ax[1].set_aspect('equal', adjustable='box')
ax[1].plot([0, 1], [0, 1], 'k--')
ax[1].plot(fpr, tpr)
ax[1].set_xlabel('False Positive Rate')
ax[1].set_ylabel('True Positive Rate')
ax[1].set_title('ROC Curve')

# Plot Confusion Matrix in 3rd subplot
import seaborn as sns
cf_matrix = confusion_matrix(y_test, predictions)
group_names = ['True non-survivor','False survivor','False non-survivor','True survivor']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
hmap = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues', ax=ax[2], square=True)
hmap.set_title('Confusion Matrix\n')
hmap.set_xlabel('Predicted Values')
hmap.set_ylabel('Actual Values')
hmap.xaxis.set_ticklabels(['non-survivor','survivor'])
hmap.yaxis.set_ticklabels(['non-survivor','survivor'])

# Plot metrics in 4th subplot
f_accuracy_logres = accuracy_score(y_test, predictions)
f_recall_logres = recall_score(y_test, predictions)
f_precision_logres = precision_score(y_test, predictions)
f_f1score = f1_score(y_test, predictions)
f_auc = roc_auc_score(y_test, y_scores[:,1])
s_auc = '{:.4f}'.format(f_auc) # save the string to be used as part of the JPG name
fontsize = 14
padding_left_Attrib = 0.15
padding_left_Value = 0.45
padding_top = 0.66
padding_top_increment = 0.07
ax[3].axis("off")
ax[3].text(padding_left_Attrib, padding_top, 'Accuracy:', size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Value, padding_top, '{:.4f}'.format(f_accuracy_logres), size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Attrib, padding_top-padding_top_increment, 'Recall:', size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Value, padding_top-padding_top_increment, '{:.4f}'.format(f_recall_logres), size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Attrib, padding_top-2*padding_top_increment, 'Precision:', size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Value, padding_top-2*padding_top_increment, '{:.4f}'.format(f_precision_logres), size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Attrib, padding_top-3*padding_top_increment, 'F1 Score:', size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Value, padding_top-3*padding_top_increment, '{:.4f}'.format(f_f1score), size=fontsize, va="top", ha="left")
ax[3].text(padding_left_Attrib, padding_top-4*padding_top_increment, 'AUC:', size=fontsize, va='top', ha='left')
ax[3].text(padding_left_Value, padding_top-4*padding_top_increment, s_auc, size=fontsize, va='top', ha='left')

# Save plot as JPG file in output folder
# from datetime import datetime
# import os
# dt_string = datetime.now().strftime("%Y-%m-%d %H%M%S") # for prefixing datetime to file name
# filename = dt_string+' '+'Logistic Regresion'+', AUC - '+s_auc+'.jpg' # AUC metric tag serves as a quick comparison between runs while seeing in the folder
# parentdirectory = os.path.abspath(os.getcwd()) # gets directory this script is run in
# outputdirectory = os.path.join(parentdirectory, 'output') # place JPGs in output folder one directory above where script is
# fig.savefig(os.path.join(outputdirectory, filename), bbox_inches='tight', dpi=300)

The pre-processing (scaling and one hot encoding) has improved the model's prediction.

Accuracy: 0.7906 -> 0.8387

Recall: 0.5530 -> 0.6841

Precision 0.7302 -> 0.7865

F1 Score: 0.6294 -> 0.7317

AUC: 0.8319 -> 0.8731
***
# 6. Pipeline creation with refactored code
We can test individual features to see how they singly contribute to the predictions, and also see the performance of other classification algorithms.

We will use re-factored code from the 'src' subdirectory:

In [ ]:
from src.pipeline_classifier import *

i_algo = 1
# Enter integer 1 to 4 to select the ML algorithm to train and test the model on.
# 1 - Logistic Regression
# 2 - Random Forest Classifier
# 3 - Support Vector Machine
# 4 - K-Nearest Neighbour

f_testsize = 0.3
# Enter a float to set the train-test size
# e.g. 0.3 -> 30% of data is reserved for testing

i_seed = 42
# Enter the seed for the random state shuffling during train-test data split (useful for reproducibility).
# Set to <None> to allow randomisation across different runs.

ls_features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
# Enter the features you want as a list.
# ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']

fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

***
### 6a. Experimentation and analysis of features on predictive ability
Let's examine the impact of individual features on the predictions.

In [ ]:
i_algo = 1
f_testsize = 0.3
i_seed = 42

features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
for x in features:
    ls_features = [x]
    fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# ls_features = ['Gender']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Smoke']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Diabetes']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Age']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Ejection Fraction']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Sodium']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Creatinine']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Platelets']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Creatine phosphokinase']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Blood Pressure']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Hemoglobin']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Height']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Weight']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)
# ls_features = ['Favorite color']
# fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed)

Observation on the predictive ability of the Logistic Regression classification model just run:

Gender, Smoke, Diabetes, Creatine phosphokinase, Height and Favorite Color contribute the least to the model's predictive ability.

Ejection Fraction, Platelets, Blood Pressure, and Hemoglobin contribute a little.

While Age, Sodium, Creatinine, and Weight have the largest effect on predictive ability.

We also evidently see that a high Precision does not mean much if the model predicts everyone as survivors.

Intuitively, we would expect Favorite Color to have zero impact but its AUC is higher than even Gender, Smoke, Diabetes, Creatine Phosphokinase and Height.

We could change the test size higher and also change the random_state seed.

In [ ]:
i_algo = 1
f_testsize = 0.4
i_seed = 3
ls_features = ['Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

The AUC is lower in this run. We can pretty much say it suggests no discrimimation.
***
We can try a combination of features next.

In [ ]:
i_algo = 1
f_testsize = 0.3
i_seed = 42

# identified least impactful features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Creatine phosphokinase', 'Height', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderately impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified most impactful features
ls_features = ['Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderate and most impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin', 'Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# all features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

***
### 6a. Experimentation and analysis of different algorithms on predictive ability
We train and evaluate with Random Forest Classifier next.

In [ ]:
i_algo = 2
f_testsize = 0.3
i_seed = 42

# identified least impactful features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Creatine phosphokinase', 'Height', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderately impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified most impactful features
ls_features = ['Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderate and most impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin', 'Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# all features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

The Random Forest Classifier model is excellent!
***
We train and evaluate with Support Vector Machine next. (this may take 6 to 7 minutes to run)

In [ ]:
i_algo = 3
f_testsize = 0.3
i_seed = 42

# identified least impactful features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Creatine phosphokinase', 'Height', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderately impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified most impactful features
ls_features = ['Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderate and most impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin', 'Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# all features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

For the SVM model, the performance increases significantly with more features.
***
We train and evaluate with K-Nearest Neighbour next.

In [ ]:
i_algo = 4
f_testsize = 0.3
i_seed = 42

# identified least impactful features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Creatine phosphokinase', 'Height', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderately impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified most impactful features
ls_features = ['Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# identified moderate and most impactful features
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin', 'Age', 'Sodium', 'Creatinine', 'Weight']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

# all features
ls_features = ['Gender', 'Smoke', 'Diabetes', 'Age', 'Ejection Fraction', 'Sodium', 'Creatinine', 'Platelets', 'Creatine phosphokinase', 'Blood Pressure', 'Hemoglobin', 'Height', 'Weight', 'Favorite color']
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], i_algo, f_testsize, i_seed, True, False)

Similarly the KNN model, the performance increases significantly with more features. The jump in predictive ability with just the earlier identified moderate features is more than with the SVM model.
***
Let's compare the performance contributed by the moderate features across the 4 algorithms.

In [ ]:
# identified moderately impactful features
f_testsize = 0.3
i_seed = 42
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin']

fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 1, f_testsize, i_seed, True, False)
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 2, f_testsize, i_seed, True, False)
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 3, f_testsize, i_seed, True, False)
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 4, f_testsize, i_seed, True, False)

Here we can clearly see that the model trained on the Random Forest Classifier algorithm performs the best.
***
For completeness, let's compare the performance contributed by the most impactful features across the 4 algorithms.

In [ ]:
# identified most impactful features
f_testsize = 0.3
i_seed = 42
ls_features = ['Ejection Fraction', 'Platelets', 'Blood Pressure', 'Hemoglobin', 'Age', 'Sodium', 'Creatinine', 'Weight']

fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 1, f_testsize, i_seed, True, False)
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 2, f_testsize, i_seed, True, False)
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 3, f_testsize, i_seed, True, False)
fig = pipeline_classifier(ls_features, df[ls_features], df['Survive'], 4, f_testsize, i_seed, True, False)

***
# 7. Conclusion

We see that the model trained on Random Forest Classifier being the best predictor followed by K-Nearest Neighbour.

I reckon by virtue of its ability to correct for overfitting helps it outperform the other 3 algorithms tested.

Further testing should be performed with unseen data to further increase confidence in the model.